<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/covid_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка шаблонов

Шаблон табличной сводки по миру - файл xlsx

In [6]:
from google.colab import files
world_input_file = files.upload()
world_input_file = next(iter(world_input_file))

Табличня сводка по россии - файл xlsx

In [7]:
from google.colab import files
russia = files.upload()
russia = next(iter(russia))

Saving russia_regions.xlsx to russia_regions (1).xlsx


#Подготовка данных с репозитория университета Хопкинса

In [8]:
![ ! -d "/content/COVID-19/" ] && git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 167498, done.
remote: Counting objects: 100% (1142/1142), done.
remote: Compressing objects: 100% (626/626), done.
remote: Total 167498 (delta 536), reused 1106 (delta 513), pack-reused 166356
Receiving objects: 100% (167498/167498), 1.57 GiB | 28.16 MiB/s, done.
Resolving deltas: 100% (126633/126633), done.
Checking out files: 100% (1238/1238), done.


In [9]:
from openpyxl.worksheet.worksheet import Worksheet
import pandas as pd
import glob
from datetime import datetime, timedelta
from copy import copy
import openpyxl
from openpyxl.formula.translate import Translator
from openpyxl.utils.cell import get_column_letter
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows

WEEKLY_REPORT_DAY = 1
countries_map = {"Taiwan*":"Тайвань", "Timor-Leste":"Восточный Тимор", "US":"США", "West Bank and Gaza":"Западный берег", "Syria": "Сирия", "Slovakia": "Словакия", "Sao Tome and Principe": "Сан-Томе и Принсипи", "Russia": "Россия", "Micronesia": "Федеративные Штаты Микронезии", "Korea, South": "Южная Корея", "Iran": "Иран", "Eswatini": "Эсватини", "Cote d'Ivoire": "Кот-д'Ивуар", "Congo (Kinshasa)": "ДР Конго", "Congo (Brazzaville)":"Браззавиль", "Cabo Verde":"Кабо-Верде", "Burma": "Мьянма", "Afghanistan":  "Афганистан", "Albania":  "Албания", "Algeria":  "Алжир", "American Samoa":  "Американское Самоа", "Andorra":  "Андорра", "Angola":  "Ангола", "Anguilla":  "Ангилья", "Antarctica":  "Антарктика", "Antigua and Barbuda":  "Антигуа и Барбуда", "Argentina":  "Аргентина", "Armenia":  "Армения", "Aruba":  "Аруба", "Australia":  "Австралия", "Austria":  "Австрия", "Azerbaijan":  "Азербайджан", "Bahamas":  "Багамские Острова", "Bahrain":  "Бахрейн", "Bangladesh":  "Бангладеш", "Barbados":  "Барбадос", "Belarus":  "Белоруссия", "Belgium":  "Бельгия", "Belize":  "Белиз", "Benin":  "Бенин", "Bermuda":  "Бермудские Острова", "Bhutan":  "Бутан", "Bolivia":  "Боливия", "Bosnia and Herzegovina":  "Босния и Герцеговина", "Botswana":  "Ботсвана", "Brazil":  "Бразилия", "Brunei":  "Бруней", "Bulgaria":  "Болгария", "Burkina Faso":  "Буркина-Фасо", "Burundi":  "Бурунди", "Cambodia":  "Камбоджа", "Cameroon":  "Камерун", "Canada":  "Канада", "Cape Verde":  "Кабо-Верде", "Cayman Islands":  "Острова Кайман", "Central African Republic":  "Центральноафриканская Республика", "Chad":  "Чад", "Chile":  "Чили", "China":  "Китай", "Colombia":  "Колумбия", "Comoros":  "Коморские острова", "Democratic Republic of the Congo (Kinshasa)":  "Демократическая Республика Конго", "Congo, Republic of (Brazzaville)":  "Республика Конго", "Cook Islands":  "Острова Кука", "Costa Rica":  "Коста-Рика", "Ivory Coast (Cote d'Ivoire)":  "Кот-д'Ивуар", "Croatia":  "Хорватия", "Cuba":  "Куба", "Cyprus":  "Кипр (остров)", "Czechia":  "Чехия", "Denmark":  "Дания", "Djibouti":  "Джибути", "Dominica":  "Доминика", "Dominican Republic":  "Доминиканская Республика", "East Timor (Timor-Leste)":  "Восточный Тимор", "Ecuador":  "Эквадор", "Egypt":  "Египет", "El Salvador":  "Сальвадор", "Equatorial Guinea":  "Экваториальная Гвинея", "Eritrea":  "Эритрея", "Estonia":  "Эстония", "Ethiopia":  "Эфиопия", "Falkland Islands":  "Фолклендские острова", "Faroe Islands":  "Фарерские острова", "Fiji":  "Фиджи", "Finland":  "Финляндия", "France":  "Франция", "French Guiana":  "Гвиана", "French Polynesia":  "Французская Полинезия", "Gabon":  "Габон", "Gambia":  "Гамбия", "Georgia":  "Грузия", "Germany":  "Германия", "Ghana":  "Гана", "Gibraltar":  "Гибралтар", "Greece":  "Греция", "Greenland":  "Гренландия", "Grenada":  "Гренада", "Guadeloupe":  "Гваделупа", "Guam":  "Гуам", "Guatemala":  "Гватемала", "Guinea":  "Гвинея", "Guinea-Bissau":  "Гвинея-Бисау", "Guyana":  "Гайана", "Haiti":  "Гаити", "Honduras":  "Гондурас", "Hong Kong":  "Гонконг", "Hungary":  "Венгрия", "Iceland":  "Исландия", "India":  "Индия", "Indonesia":  "Индонезия", "Iran (Islamic Republic of)":  "Иран (Персия)", "Iraq":  "Ирак", "Ireland":  "Ирландия", "Israel":  "Израиль", "Italy":  "Италия", "Jamaica":  "Ямайка", "Japan":  "Япония", "Jordan":  "Иордания", "Kazakhstan":  "Казахстан", "Kenya":  "Кения", "Kiribati":  "Кирибати", "Korea, Democratic People's Rep. (North Korea)":  "Северная Корея (КНДР)", "Korea, Republic of (South Korea)":  "Ю́жная Коре́я (Республика Корея)", "Kosovo":  "Косово (Республика Косово)", "Kuwait":  "Кувейт", "Kyrgyzstan":  "Киргизия", "Laos":  "Лаос", "Latvia":  "Латвия", "Lebanon":  "Ливан", "Lesotho":  "Лесото", "Liberia":  "Либерия", "Libya":  "Ливия", "Liechtenstein":  "Лихтенштейн", "Lithuania":  "Литва", "Luxembourg":  "Люксембург", "Macau":  "Макао (Аомынь)", "North Macedonia":  "Северная Македония", "Madagascar":  "Мадагаскар", "Malawi":  "Малави", "Malaysia":  "Малайзия", "Maldives":  "Мальдивы", "Mali":  "Мали", "Malta":  "Мальта", "Marshall Islands":  "Маршалловы Острова", "Martinique":  "Мартиника", "Mauritania":  "Мавритания", "Mauritius":  "Маврикий", "Mayotte":  "Майотта", "Mexico":  "Мексика", "Микронезия, Federated States of":  "Федеративные Штаты Микронезии", "Moldova":  "Молдавия", "Monaco":  "Монако, Княжество Монако", "Mongolia":  "Монголия", "Montenegro":  "Черногория", "Montserrat":  "Монтсеррат", "Morocco":  "Марокко", "Mozambique":  "Мозамбик", "Myanmar, Burma":  "Мьянма, (Би́рма)", "Namibia":  "Намибия", "Nauru":  "Науру", "Nepal":  "Непал", "Netherlands":  "Нидерланды, Голландия", "New Caledonia":  "Новая Каледония", "New Zealand":  "Новая Зеландия", "Nicaragua":  "Никарагуа", "Niger":  "Нигер", "Nigeria":  "Нигерия", "Northern Mariana Islands":  "Северные Марианские", "":  "острова", "Norway":  "Норвегия", "Oman":  "Оман", "Pakistan":  "Пакистан", "Palau":  "Палау", "State of Palestine":  "Государство Палестина", "Panama":  "Панама", "Papua New Guinea":  "Папуа — Новая Гвинея", "Paraguay":  "Парагвай", "Peru":  "Перу", "Philippines":  "Филиппины", "Poland":  "Польша", "Portugal":  "Португалия", "Puerto Rico":  "Пуэрто-Рико", "Qatar":  "Катар", "Réunion":  "Реюньон", "Romania":  "Румыния", "Russian Federation":  "Россия, Росси́йская Федера́ция", "Rwanda":  "Руанда", "Saint Kitts and Nevis":  "Сент-Китс и Невис", "Saint Lucia":  "Сент-Люсия", "Saint Vincent and the Grenadines":  "Сент-Винсент и Гренадины", "Samoa":  "Самоа", "San Marino":  "Сан-Марино", "Sao Tome and Príncipe":  "Сан-Томе и Принсипи", "Saudi Arabia":  "Саудовская Аравия", "Senegal":  "Сенегал", "Serbia":  "Сербия", "Seychelles":  "Сейшельские Острова", "Sierra Leone":  "Сьерра-Леоне", "Singapore":  "Сингапур", "Slovakia (Slovak Republic)":  "Словакия", "Slovenia":  "Словения", "Solomon Islands":  "Соломоновы Острова", "Somalia":  "Сомали", "South Africa":  "Южная Африка", "South Sudan":  "Южный Судан", "Spain":  "Испания", "Sri Lanka":  "Шри-Ланка (ранее Цейлон)", "Sudan":  "Судан", "Suriname":  "Суринам", "Swaziland (Eswatini)":  "Сва́зиле́нд (Эсватини)", "Sweden":  "Швеция", "Switzerland":  "Швейцария", "Syria, Syrian Arab Republic":  "Сирия", "Taiwan (Republic of China)":  "Китайская Республика (Тайвань)", "Tajikistan":  "Таджикистан", "Tanzania":  "Танзания", "Thailand":  "Таиланд (ранее Сиам)", "Tibet":  "Тибет", "Timor-Leste (East Timor)":  "Восточный Тимор", "Togo":  "Того", "Tokelau":  "Токелау", "Tonga":  "Тонга", "Trinidad and Tobago":  "Тринидад и Тобаго", "Tunisia":  "Тунис", "Turkey":  "Турция", "Turkmenistan":  "Туркмения", "Tuvalu":  "Тувалу", "Uganda":  "Уганда", "Ukraine":  "Украина", "United Arab Emirates":  "Объединённые Арабские Эмираты (ОАЭ)", "United Kingdom":  "Великобритания (Соединённое Короле́вство)", "United States":  "Соединённые Штаты Америки (Аме́рика, США)", "Uruguay":  "Уругвай", "Uzbekistan":  "Узбекистан", "Vanuatu":  "Вануату", "Vatican City State (Holy See)":  "Ватикан (Святой Престол)", "Venezuela":  "Венесуэла", "Vietnam":  "Вьетнам", "Virgin Islands (British)":  "Виргинские Острова (Великобритания)", "Virgin Islands (U.S.)":  "Виргинские Острова (США)", "Wallis and Futuna Islands":  "Уоллис и Футуна", "Western Sahara":  "Западная Сахара", "Yemen":  "Йемен", "Zambia":  "Замбия", "Zimbabwe":  "Зимбабве"}

In [10]:
path = r'/content/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename_csv in all_files:
    df = pd.read_csv(filename_csv, index_col=None, header=0)
    df['Last_Update'] = filename_csv.split('.')[0].split('/')[-1]
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame['Last_Update'] = pd.to_datetime(frame['Last_Update'], format='%m-%d-%Y')

In [11]:
def translate_country(eng):
    try:
        return countries_map[eng]
    except KeyError:
        return eng


frame['Country_Region'] = frame['Country_Region'].apply(translate_country)

In [12]:
df = frame[['Confirmed', 'Recovered', 'Deaths', 'Country_Region', 'Last_Update', 'Combined_Key']]
sorted_df = df.groupby(['Last_Update', 'Country_Region']).sum().reset_index().sort_values(by=['Country_Region', 'Last_Update'])


In [13]:
li = []
for country in sorted_df['Country_Region'].unique():
    country_df = sorted_df[sorted_df['Country_Region'] == country].copy()
    country_df_diff = country_df[['Confirmed', 'Recovered', 'Deaths']].diff()
    country_df['Confirmed_diff'] = country_df_diff['Confirmed']
    country_df['Recovered_diff'] = country_df_diff['Recovered']
    country_df['Deaths_diff'] = country_df_diff['Deaths']
    li.append(country_df)


df_diff = pd.concat(li, axis=0, ignore_index=True)
df_diff = df_diff.fillna(0)
df_diff = df_diff[['Country_Region', 'Last_Update', 'Confirmed', 'Recovered', 'Deaths', 'Confirmed_diff', 'Recovered_diff', 'Deaths_diff']]
# df_diff.to_csv('counties.csv', index=False, header=False)

#Заполнение таблиц

In [14]:
def base_countries_sheet(ws: Worksheet):
    for i, row in enumerate(dataframe_to_rows(df_diff, index=False, header=False)):
        ws.append(row)
        r = i + 2
        ws.cell(row=r, column=9).value = fr'=IF(YEAR(B{r})=2020,(IF(J{r}=1,WEEKNUM(B{r},2),IF(J{r}=2,WEEKNUM(B{r},2),IF(J{r}=3,WEEKNUM(B{r},2),IF(J{r}=4,WEEKNUM(B{r},2),IF(J{r}=5,WEEKNUM(B{r},2),IF(J{r}=6,WEEKNUM(B{r},2),IF(J{r}=7,WEEKNUM(B{r},2)+1)))))))),IF(YEAR(B{r})=2021,(IF(J{r}=1,WEEKNUM(B{r},2),IF(J{r}=2,WEEKNUM(B{r},2),IF(J{r}=3,WEEKNUM(B{r},2),IF(J{r}=4,WEEKNUM(B{r},2),IF(J{r}=5,WEEKNUM(B{r},2),IF(J{r}=6,WEEKNUM(B{r},2),IF(J{r}=7,WEEKNUM(B{r},2)+1))))))))+52))'
        ws.cell(row=r, column=10).value = fr'=WEEKDAY(B{r},2)'
        ws.cell(row=r, column=2).number_format = 'DD.MM.YYYY'

In [15]:
def continue_formula_n_down(ws: Worksheet, columns: list, row: int, n: int):
    for column in columns:

        formula = ws.cell(row=(row - n), column=column).value
        # print(row-n, column, formula)
        last_cell_id = get_column_letter(column) + str(row - n)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id] = Translator(formula, origin=last_cell_id).translate_formula(next_cell_id)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)


def continue_formula_down(ws: Worksheet, columns: list, row: int):
    continue_formula_n_down(ws, columns, row, 1)

In [16]:
def  continue_date_down(ws: Worksheet, rows: list):
    for row in rows:
        try:
            column = [c.value for c in ws[row]][3:].index(None) + 4
        except ValueError:
            column = ws.max_column + 1
        last_table_date = ws.cell(row=row, column=column - 1).value
        ws.cell(row=row, column=column).value = (last_table_date) + timedelta(days=1)
        ws.cell(row=row, column=column).number_format = 'DD.MM.YY'

        last_cell_id = get_column_letter(column - 1) + str(row)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

In [17]:
def continue_number_down(ws: Worksheet, columns: list):
    for column in columns:

        row = ws.max_row + 1
        last_num = ws.cell(row=row - 1, column=column).value
        ws.cell(row=row, column=column).value = last_num + 1

        last_cell_id = get_column_letter(column) + str(row-1)
        next_cell_id = get_column_letter(column) + str(row)
        ws[next_cell_id]._style = copy(ws[last_cell_id]._style)

In [18]:
def base_world_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    ws.cell(row=new_row, column=1).value = datetime.now()
    ws.cell(row=new_row, column=1).number_format = 'DD.MM.YY'
    continue_formula_down(ws, list(range(2, 6)) + [7, 8, 10, 11] + list(range(13, 21)), new_row)

    today = (datetime.today() - timedelta(days=1))
    daily_df = df_diff.groupby('Last_Update').sum().loc[today.strftime('%d.%m.%y')]
    ws.cell(row=new_row, column=6).value = daily_df['Confirmed']
    ws.cell(row=new_row, column=9).value = daily_df['Recovered']
    ws.cell(row=new_row, column=12).value = daily_df['Deaths']

In [19]:
def weeks_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    # print(new_row)
    continue_number_down(ws, [1])
    continue_formula_down(ws, list(range(2, 15)), new_row)


In [20]:
def countrie_sheet(ws: Worksheet):
    new_row = ws.max_row + 1
    # print(new_row)
    continue_number_down(ws, [1])
    continue_formula_down(ws, list(range(2, 32)), new_row)

In [21]:
world_out_file = 'world_out.xlsx'
def main(filename: str):
    wb = openpyxl.load_workbook(filename)

    base_countries_sheet(wb['База страны'])
    base_world_sheet(wb['База мир'])

    if (datetime.today().weekday() + 1 == WEEKLY_REPORT_DAY):
        weeks_sheet(wb['Недели'])
        for country in wb.sheetnames[8:-1]:
            countrie_sheet(wb[country])

    wb.save(world_out_file)

In [22]:
main(world_input_file)

#Результат

In [23]:
files.download(world_out_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Сводка word

In [24]:
!pip install python-docx
!pip install pip install excel-formulas-calculator

     |████████████████████████████████| 5.6 MB 7.6 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=1e291682a25deb9e063cb0476cf201c06882026ab9c8c8f6a5d661c71130f6f5
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [25]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
!pip install requests-html
!pip install xlutils

     |████████████████████████████████| 904 kB 7.8 MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubu

     |████████████████████████████████| 55 kB 2.7 MB/s 


In [26]:
from efc.interfaces.iopenpyxl import OpenpyxlInterface
from collections import defaultdict
import numpy as np

In [28]:
wb_world = openpyxl.load_workbook(world_out_file)
wb_russia = openpyxl.load_workbook(russia)

In [29]:
interface_world = OpenpyxlInterface(wb=wb_world, use_cache=True)
interface_russia = OpenpyxlInterface(wb=wb_russia, use_cache=True)

In [30]:
def choise(var: int, s1, s2, s3):
    if var > 0:
        return s1
    elif var < 0:
        return s2
    else:
        return s3

In [31]:
def paragraph1(ws: Worksheet, doc, interface):
    row = ws.max_row
    week_to_week = int(interface.calc_cell(f'P{row}', 'База мир')*100)

    w1 = choise(week_to_week,
                    'ускоряется',
                    'замедляется',
                    'остается прежним')

    par = f'Темп роста количества новых случаев заражения {w1} (прирост за последние 7 дней находится на уровне {week_to_week}%).'
    doc.add_paragraph(par, style='List Bullet')

In [32]:
def paragraph2(ws: Worksheet, doc, interface):
    row = [c.value for c in ws['AC'][7:]].index(None) + 7
    week = interface.calc_cell(f'AC{row}', 'База РФ')
    week_to_week = int(interface.calc_cell(f'AD{row}', 'База РФ')*100)
    week_to_week_prev = int(interface.calc_cell(f'AD{row-1}', 'База РФ')*100)

    w1 = choise(week_to_week,
                    'наблюдается динамика увеличения заболеваемости',
                    'наблюдается динамика уменьшения заболеваемости',
                    'динамика заболеваемости остается на прежнем уровне')

    if week_to_week < week_to_week_prev:
        w2 = 'замедлились'
    elif week_to_week > week_to_week_prev:
        w2 = 'ускорились'
    else:
        w2 = 'остались неизменными'

    dynamic = interface.calc_cell(f'E{row}', 'База РФ')
    dynamic_speed = int(interface.calc_cell(f'R{row}', 'База РФ')*100)
    dynamic_speed_prev = int(interface.calc_cell(f'R{row-1}', 'База РФ')*100)

    w3 = choise(dynamic_speed,
                f'повысился на {abs(dynamic_speed)}%',
                f'снизился на {abs(dynamic_speed)}%',
                'остался неизменным')
    w4 = choise(dynamic_speed_prev,
                f'повысился на {abs(dynamic_speed_prev)}%',
                f'снизился на {abs(dynamic_speed_prev)}%',
                'остался неизменным')

    recover = interface.calc_cell(f'AE{row}', 'База РФ')
    recover_week_to_week = int(interface.calc_cell(f'AF{row}', 'База РФ')*100)

    mortality = interface.calc_cell(f'AG{row}', 'База РФ')
    mortality_week_to_week = int(interface.calc_cell(f'AH{row}', 'База РФ')*100)

    w5 = choise(recover_week_to_week,
                f'вырасло на {abs(dynamic_speed)}%',
                f'снизилось на {abs(dynamic_speed)}%',
                'осталось неизменным')
    
    w6 = choise(mortality_week_to_week,
                f'вырос на {abs(dynamic_speed)}%',
                f'снизился на {abs(dynamic_speed)}%',
                'остался неизменным')

    day_deaths = interface.calc_cell(f'H{row}', 'База РФ')
    max_day_deaths = interface.calc_cell(f'L{row}', 'База РФ')
    is_daily_record = ', что является новым суточным максимумом за весь период пандемии.' if day_deaths == max_day_deaths else '.' 

    par = f'В России {w1}, темпы недельного роста {w2}: количество заболевших за последние 7 дней увеличилось на {week_to_week}% (аналогичный показатель днем ранее - {week_to_week_prev}%) и составило {week} случаев. '\
        f'Темп суточной динамики {w3} (днем ранее {w4}), за истекшие сутки зарегистрировано {dynamic} случаев(ая). '\
        f'Количество выздоровлений за семидневный период {w5} по сравнению с показателем предшествующей недели и составило {recover} случаев(ая). '\
        f'Недельный прирост количества летальных исходов {w6} и составил {mortality} случаев(ая). '\
        f'Суточный показатель составил {day_deaths} случаев(ая){is_daily_record}'
    doc.add_paragraph(par, style='List Bullet')

In [33]:
def paragraph3(ws: Worksheet, doc, interface):
    col = ws.max_column 
    gain = interface.calc_cell(f'{get_column_letter(col)}3', 'МСК и СП')
    gain_speed = int(interface.calc_cell(f'{get_column_letter(col)}6', 'МСК и СП')*100)
    gain_speed_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}6', 'МСК и СП')*100)

    gain_week = int(interface.calc_cell(f'{get_column_letter(col)}13', 'МСК и СП')*100)
    gain_week_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}13', 'МСК и СП')*100)

    w1 = choise(gain_speed,
                'зафиксировано повышение суточной динамики',
                'зафиксировано снижение суточной динамики',
                'суточный прирост не изменился')
    
    w2 = choise(gain_speed,
                f'повысился на {abs(gain_speed)}%',
                f'снизился на {abs(gain_speed)}%',
                f'остался неизменным')
    
    w3 = choise(gain_speed_prev,
                f'повышение составило {gain_speed_prev}%',
                f'снижение составило {gain_speed_prev}%',
                f'темп остался неизменным')
    
    w4 = choise(gain_week,
                f'повышения недельного прироста составляет {abs(gain_week)}%',
                f'снижения недельного прироста составляет {abs(gain_week)}%',
                f'недельного прироста остался неизменным')
    
    w5 = choise(gain_week_prev,
                f'повышениe {abs(gain_week_prev)}%',
                f'снижениe {abs(gain_week_prev)}%',
                f'темп был неизменным')

    par = f'В Москве {w1}: за истекший день темп прироста {w2} (днем ранее {w3}), число выявленных случаев составило {gain}. '\
        f'Темп {w4} (днем ранее {w5}).'

    doc.add_paragraph(par, style='List Bullet')



In [34]:
def paragraph4(ws: Worksheet, doc, interface):
    col = ws.max_column 
    gain = interface.calc_cell(f'{get_column_letter(col)}4', 'МСК и СП')
    gain_speed = int(interface.calc_cell(f'{get_column_letter(col)}7', 'МСК и СП')*100)
    gain_speed_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}7', 'МСК и СП')*100)

    gain_week = int(interface.calc_cell(f'{get_column_letter(col)}15', 'МСК и СП')*100)
    gain_week_prev = int(interface.calc_cell(f'{get_column_letter(col-1)}15', 'МСК и СП')*100)

    w1 = choise(gain_speed,
                'зафиксировано повышение суточной динамики',
                'зафиксировано снижение суточной динамики',
                'суточный прирост не изменился')
    
    w2 = choise(gain_speed,
                f'повысился на {abs(gain_speed)}%',
                f'снизился на {abs(gain_speed)}%',
                f'остался неизменным')
    
    w3 = choise(gain_speed_prev,
                f'повышение составило {gain_speed_prev}%',
                f'снижение составило {gain_speed_prev}%',
                f'темп остался неизменным')
    
    w4 = choise(gain_week,
                f'повышения недельного прироста составляет {abs(gain_week)}%',
                f'снижения недельного прироста составляет {abs(gain_week)}%',
                f'недельного прироста остался неизменным')
    
    w5 = choise(gain_week_prev,
                f'повышениe {abs(gain_week_prev)}%',
                f'снижениe {abs(gain_week_prev)}%',
                f'темп был неизменным')

    par = f'В Санкт-Петербурге {w1}: за истекший день темп прироста {w2} (днем ранее {w3}), число выявленных случаев составило {gain}. '\
        f'Темп {w4} (днем ранее {w5}).'

    doc.add_paragraph(par, style='List Bullet')


In [35]:
def paragraph5(ws: Worksheet, doc, interface):
    info = defaultdict(list)
    col = ws.max_column
    # cities = []
    # percents = []
    for i in range(2, 87):
        info['ratio'].append(interface.calc_cell(f'{get_column_letter(col)}{i}', 'прирост 7дн'))
        info['regions'].append(interface.calc_cell(f'{get_column_letter(col-1)}{i}', 'прирост 7дн'))
        info['cases'].append(interface.calc_cell(f'{get_column_letter(col-3)}{i}', 'прирост 7дн'))
    top5_idx = np.array(info['cases']).argsort()[-5:][::-1]
    top5_regions = np.array(info['regions'])[top5_idx]
    top5_cases = np.array(info['cases'])[top5_idx]

    total = interface.calc_cell(f'{get_column_letter(col-3)}{87}', 'прирост 7дн')
    top5_percents = [case/total for case in top5_cases]

    top5_list = [f'{region} ({round(ratio*100)}%)' for region, ratio in zip(top5_regions, top5_percents)]
    # print(top5_list)
    par = f'Регионы, которые обеспечили основной прирост по России за последние 7 дней:\n{", ".join(top5_list)}.'


    doc.add_paragraph(par, style='List Bullet')

In [36]:
def paragraph6(ws: Worksheet, doc, interface):
    info = defaultdict(list)
    col = ws.max_column
    # cities = []
    # percents = []
    for i in range(2, 87):
        info['ratio'].append(interface.calc_cell(f'{get_column_letter(col)}{i}', 'прирост 7дн'))
        info['regions'].append(interface.calc_cell(f'{get_column_letter(col-1)}{i}', 'прирост 7дн'))
        info['last_date'].append(interface.calc_cell(f'{get_column_letter(col-3)}{i}', 'прирост 7дн'))
    top20 = np.array(info['last_date']).argsort()[-20:][::-1]
    top20_regions = np.array(info['regions'])[top20]
    top20_ratio = np.array(info['ratio'])[top20]
    top5_ratio_index = top20_ratio.argsort()[-5:][::-1]
    top5_regions = top20_regions[top5_ratio_index]
    top5_ratio = top20_ratio[top5_ratio_index]
    top5_list = [f'{region} (рост на {round(ratio*100)}%)' for region, ratio in zip(top5_regions, top5_ratio)]
    # print(top5_list)

    bigger50_idx = [i for i, value in enumerate(info['ratio']) if round(value*100) >= 50]
    bigger50_regions = np.array(info['regions'])[bigger50_idx]

    

    if len(bigger50_regions) > 0:
        par2 = f'\nСтоит также отметить, что дополнительно к 5 названным еще в {len(bigger50_regions)} регионе(ах) с меньшей статистикой* скорость недельного прироста превышает 50%.\n'\
                f'* {", ".join(bigger50_regions)}'
    else:
        par2 = ''
    
    par1 = f'Скорость прироста от недели к неделе (из числа ТОП-20 по абсолютному приросту за последние 7 дней): {", ".join(top5_list)}.'
    doc.add_paragraph(par1+par2, style='List Bullet')

In [37]:
def fill_header(doc):
    header = '[В Мире и России]'
    description = 'Файл «МИР».\n'\
                    'Лист «Источник» - ссылка на исходные данные. Скачиваем данные в формате xls. Загружаем в лист «База страны».\n'\
                    'Лист «База мир». В столбце А проставить текущую дату, протянуть формулы по всей строке.\n'\
                    'Лист «Сутки_забол». Задаем формулы для формирования данных за сутки и за последние 7 дней. Дата для формирования данных за сутки задается как Т-1 (дата предыдущего дня).\n'\
                    'Данные по вакцинации берем с сайта GOGOV.ru вакцинация. Других источников таких данных пока не обнаружили.\n'\
                    'Иногда случаются «выбросы» по отдельным странам, когда пересматривается подсчет показателей и за один день показывается статистика за целый период. В этом случае ищем «выброс» в листе «База страны», по какой стране произошло нестандартное отклонение.'
    p1 = doc.add_paragraph()
    p1.alignment = 1
    font = p1.add_run(header).font
    font.color.rgb = RGBColor.from_string('EB4034')
    
    p2 = doc.add_paragraph()
    font = p2.add_run(description).font
    font.color.rgb = RGBColor.from_string('A32E26')



In [38]:
!pip install html2text

In [39]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [41]:
import html2text

In [42]:
RBC_URL = 'https://www.rbc.ru/story/5e2881539a794724ab627caa'
wd.get(RBC_URL)
news_links = wd.find_elements_by_xpath("//a[@class='item__link']")
headers = wd.find_elements_by_xpath("//span[@class='item__title rm-cm-item-text']")

for i, header in enumerate(headers):
    try:
        if header.text.split('.')[1].startswith(' Актуальное о пандемии на'):
            break
    except IndexError:
        continue

news_links[i].click()
elem = wd.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

h = html2text.HTML2Text()
h.ignore_links = True
source_text = h.handle(source_code)
rbc_news = [s[4:] for s in source_text.split('\n') if s.startswith('  * ')]

In [43]:
STOPCORONA_URL = 'https://стопкоронавирус.рф/news/'
wd.get(STOPCORONA_URL)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='cv-news-page__news-list-item']//h2")
stopcorona_headers = [header.text for header in stopcorona_headers_elements]


In [44]:
STOPCORONA_URL_BUISNESS = 'https://стопкоронавирус.рф/news/?tags=1'
wd.get(STOPCORONA_URL_BUISNESS)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='cv-news-page__news-list-item']//h2")
stopcorona_headers_buisness = [header.text for header in stopcorona_headers_elements]


In [45]:
YANDEX_URL = 'https://yandex.ru/news/rubric/koronavirus'
wd.get(YANDEX_URL)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='mg-card__annotation']")
yandex_annotation = [header.text for header in stopcorona_headers_elements if header.text != '']


In [49]:
def paragraph_news(doc):
    doc.add_paragraph('Справочно')
    p = doc.add_paragraph()
    runner = p.add_run('РБК')
    runner.bold = True
    for news in rbc_news:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Стопкоронавирус')
    runner.bold = True
    for news in stopcorona_headers[:10]:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Стопкоронавирус по тегу "бизнесу"')
    runner.bold = True
    for news in stopcorona_headers_buisness[:10]:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Яндекс')
    runner.bold = True
    for news in yandex_annotation[:10]:
        doc.add_paragraph(news, style='List Bullet')

In [50]:
import docx
from docx.shared import RGBColor
doc = docx.Document()
fill_header(doc)
paragraph1(wb_world['База мир'], doc, interface_world)
paragraph2(wb_russia['База РФ'], doc, interface_russia)
paragraph3(wb_russia['МСК и СП'], doc, interface_russia)
paragraph4(wb_russia['МСК и СП'], doc, interface_russia)
paragraph5(wb_russia['прирост 7дн'], doc, interface_russia)
paragraph6(wb_russia['прирост 7дн'], doc, interface_russia)
paragraph_news(doc)
doc.save('сводка.docx')


# Результат

In [51]:
files.download('сводка.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>